<a href="https://colab.research.google.com/github/Yeyeong99/Aiffel/blob/main/Going%20Deeper/Gd12_%EB%B2%88%EC%97%AD_%EB%AA%A8%EB%8D%B8%EC%9D%84_%ED%99%9C%EC%9A%A9%ED%95%9C_%EC%B1%97%EB%B4%87_%EC%A0%9C%EC%9E%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [2]:
! mkdir -p /content/drive/MyDrive/aiffel/transformer_chatbot/data

In [3]:
! pip install sentencepiece

### 데이터 로딩

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sentencepiece as spm
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from konlpy.tag import Mecab

import re
import os
import random
import math

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [5]:
raw_data = pd.read_csv('/content/drive/MyDrive/aiffel/transformer_chatbot/data/ChatbotData .csv')
raw_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


Q열을 기준으로 중복값을 제거해준다.

In [6]:
raw_data.drop_duplicates(['Q'], inplace = True)

In [7]:
len(raw_data)

11662

In [8]:
questions = raw_data['Q']
answers = raw_data['A']

print(questions[:5])
print()
print(answers[:5])

0             12시 땡!
1        1지망 학교 떨어졌어
2       3박4일 놀러가고 싶다
3    3박4일 정도 놀러가고 싶다
4            PPL 심하네
Name: Q, dtype: object

0     하루가 또 가네요.
1      위로해 드립니다.
2    여행은 언제나 좋죠.
3    여행은 언제나 좋죠.
4     눈살이 찌푸려지죠.
Name: A, dtype: object


### 데이터 정제 및 토큰화

In [9]:
def preprocess_sentence(sentence):
    
    sentence = sentence.lower()
    sentence = re.sub(r"[^0-9a-zA-Z가-힣ㄱ-ㅎㅏ-ㅣ?.!,]+", " ", sentence)

    return sentence

In [10]:
q_len = list(map(lambda x: len(x), questions))
a_len = list(map(lambda x: len(x), answers))

In [11]:
max(q_len), max(a_len)

(56, 76)

In [12]:
sum(q_len)/len(q_len), sum(a_len)/len(a_len)

(12.909963985594239, 15.032327216600926)

토큰화를 한 뒤 길이가 20을 넘는 문장은 제외한다.

In [13]:
def build_corpus(questions, answers):
    questions = list(map(lambda x: preprocess_sentence(x), questions))
    answers = list(map(lambda x: preprocess_sentence(x), answers))

    mecab = Mecab()

    q_corpus = list(map(lambda x: mecab.morphs(x), questions))
    a_corpus = list(map(lambda x: mecab.morphs(x), answers)) 

    que_corpus = []
    ans_corpus = []
    for q, a in zip(q_corpus, a_corpus): 
      if len(q) < 20 and len(a) < 20:
        que_corpus.append(q)
        ans_corpus.append(a)

    return que_corpus, ans_corpus

In [ ]:
que_corpus, ans_corpus = build_corpus(questions, answers)

In [15]:
que_corpus[:5], ans_corpus[:5]

([['12', '시', '땡', '!'],
  ['1', '지망', '학교', '떨어졌', '어'],
  ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'],
  ['3', '박', '4', '일', '정도', '놀', '러', '가', '고', '싶', '다'],
  ['ppl', '심하', '네']],
 [['하루', '가', '또', '가', '네요', '.'],
  ['위로', '해', '드립니다', '.'],
  ['여행', '은', '언제나', '좋', '죠', '.'],
  ['여행', '은', '언제나', '좋', '죠', '.'],
  ['눈살', '이', '찌푸려', '지', '죠', '.']])

In [16]:
len(questions), len(que_corpus)

(11662, 11440)

### 데이터 Augmentation

In [18]:
import gensim

file_path = '/content/drive/MyDrive/aiffel/transformer_chatbot/data/ko.bin'
ko_model = gensim.models.Word2Vec.load(file_path)
a = ko_model.wv.most_similar("강아지")

print(a)

[('고양이', 0.7290452718734741), ('거위', 0.7185635566711426), ('토끼', 0.7056223154067993), ('멧돼지', 0.6950401067733765), ('엄마', 0.6934334635734558), ('난쟁이', 0.6806551218032837), ('한마리', 0.6770296096801758), ('아가씨', 0.6750352382659912), ('아빠', 0.6729634404182434), ('목걸이', 0.6512460708618164)]


In [25]:
def lexical_sub(toks, word2vec):
    import random
    
    new_list = []

    try:
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[0][0]
        
    except:   # 단어장에 없는 단어
        return None

    for tok in toks:
        if tok is _from: new_list.append(_to)
        else: new_list.append(tok)

    return new_list

In [19]:
mecab = Mecab()
morph = [" ".join(mecab.morphs(sen)) for sen in questions]
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', num_words = 50000, oov_token="<UNK>")
tokenizer.fit_on_texts(morph)

In [22]:
ko_model.most_similar("파랑")[0][0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


'빨강'

In [27]:
from tqdm import tqdm_notebook

aug_question = list(que_corpus)
aug_answer = list(ans_corpus)
print("aug_question 데이터 수",len(aug_question))
print("aug_answer 데이터 수",len(aug_answer))

#aug 질문 추가
for idx in tqdm_notebook(range(len(que_corpus))):
    old_que = que_corpus[idx]
    
    new_que = lexical_sub(old_que, ko_model)

    if new_que is not None: 
        aug_question.append(new_que)
        aug_answer.append(ans_corpus[idx])

#aug 답변 추가
for idx in tqdm_notebook(range(len(que_corpus))):
    old_ans = ans_corpus[idx]
    
    new_ans = lexical_sub(old_ans, ko_model)

    if new_ans is not None: 
        aug_answer.append(new_ans)
        aug_question.append(que_corpus[idx])
        
print("aug_question 데이터 수",len(aug_question))
print("aug_answer 데이터 수",len(aug_answer))

aug_question 데이터 수 11440
aug_answer 데이터 수 11440


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


  0%|          | 0/11440 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/11440 [00:00<?, ?it/s]

aug_question 데이터 수 31240
aug_answer 데이터 수 31240


In [29]:
print(aug_question[-5:])
print(aug_answer[-5:])

[['후회', '없이', '사랑', '하', '고', '싶', '어'], ['훔쳐', '보', '는', '것', '도', '눈치', '보임', '.'], ['흑기사', '해', '주', '는', '짝', '남', '.'], ['힘든', '연애', '좋', '은', '연애', '라는', '게', '무슨', '차이', '일까', '?'], ['힘들', '어서', '결혼', '할까봐']]
[['진심', '으로', '들키', '보', '세요', '.'], ['티', '가', '나', '니까', '눈치', '놀드', '보이', '는', '거', '죠', '!'], ['설렜', '겠', '어요', '는데'], ['잘', '헤어질', '수', '있', '는', '사이', '여부', '인', '거', '똑같', '아요', '.'], ['도피성', '결혼', '은데', '하', '지', '않', '길', '바라', '요', '.']]


### 데이터 벡터화

In [30]:
from collections import Counter

vocab_size = 10000

words = np.concatenate(aug_question + aug_answer).tolist()
counter = Counter(words)
counter = counter.most_common(vocab_size-4)
vocab = ['', '', '', ''] + [key for key, _ in counter]
word_to_index = {word:index for index, word in enumerate(vocab)}
word_to_index

{'': 3,
 '.': 4,
 '이': 5,
 '하': 6,
 '는': 7,
 '을': 8,
 '세요': 9,
 '가': 10,
 '어': 11,
 '고': 12,
 '좋': 13,
 '거': 14,
 '해': 15,
 '있': 16,
 '보': 17,
 '은': 18,
 '지': 19,
 '?': 20,
 '도': 21,
 '나': 22,
 '아': 23,
 '게': 24,
 '겠': 25,
 '는데': 26,
 '에': 27,
 '예요': 28,
 '사람': 29,
 '어요': 30,
 '를': 31,
 '다': 32,
 '같': 33,
 '죠': 34,
 '한': 35,
 '싶': 36,
 '네요': 37,
 '없': 38,
 '사랑': 39,
 '면': 40,
 '수': 41,
 '안': 42,
 '것': 43,
 '네': 44,
 '친구': 45,
 '봐요': 46,
 '의': 47,
 '아요': 48,
 '잘': 49,
 '생각': 50,
 '말': 51,
 '할': 52,
 '마음': 53,
 '않': 54,
 '너무': 55,
 '되': 56,
 '주': 57,
 '했': 58,
 '더': 59,
 '만': 60,
 '일': 61,
 '기': 62,
 '내': 63,
 '었': 64,
 '들': 65,
 '연락': 66,
 '이별': 67,
 '남자': 68,
 '여자': 69,
 '많이': 70,
 '힘들': 71,
 '해요': 72,
 '시간': 73,
 '으면': 74,
 '남': 75,
 '길': 76,
 '에요': 77,
 '한테': 78,
 '먹': 79,
 '좀': 80,
 '썸': 81,
 '에서': 82,
 '괜찮': 83,
 '으로': 84,
 '때': 85,
 '많': 86,
 '았': 87,
 '야': 88,
 '짝': 89,
 '저': 90,
 '!': 91,
 '요': 92,
 '뭐': 93,
 '받': 94,
 '건': 95,
 '오늘': 96,
 '을까': 97,
 '그러': 98,
 '로': 99,
 '만나': 1

In [31]:
index_to_word = {index:word for word, index in word_to_index.items()}
index_to_word

{3: '',
 4: '.',
 5: '이',
 6: '하',
 7: '는',
 8: '을',
 9: '세요',
 10: '가',
 11: '어',
 12: '고',
 13: '좋',
 14: '거',
 15: '해',
 16: '있',
 17: '보',
 18: '은',
 19: '지',
 20: '?',
 21: '도',
 22: '나',
 23: '아',
 24: '게',
 25: '겠',
 26: '는데',
 27: '에',
 28: '예요',
 29: '사람',
 30: '어요',
 31: '를',
 32: '다',
 33: '같',
 34: '죠',
 35: '한',
 36: '싶',
 37: '네요',
 38: '없',
 39: '사랑',
 40: '면',
 41: '수',
 42: '안',
 43: '것',
 44: '네',
 45: '친구',
 46: '봐요',
 47: '의',
 48: '아요',
 49: '잘',
 50: '생각',
 51: '말',
 52: '할',
 53: '마음',
 54: '않',
 55: '너무',
 56: '되',
 57: '주',
 58: '했',
 59: '더',
 60: '만',
 61: '일',
 62: '기',
 63: '내',
 64: '었',
 65: '들',
 66: '연락',
 67: '이별',
 68: '남자',
 69: '여자',
 70: '많이',
 71: '힘들',
 72: '해요',
 73: '시간',
 74: '으면',
 75: '남',
 76: '길',
 77: '에요',
 78: '한테',
 79: '먹',
 80: '좀',
 81: '썸',
 82: '에서',
 83: '괜찮',
 84: '으로',
 85: '때',
 86: '많',
 87: '았',
 88: '야',
 89: '짝',
 90: '저',
 91: '!',
 92: '요',
 93: '뭐',
 94: '받',
 95: '건',
 96: '오늘',
 97: '을까',
 98: '그러',
 99: '로',
 100: '만

In [32]:
word_to_index["<pad>"] = 0
word_to_index["<start>"] = 1
word_to_index["<end>"] = 2  # unknown
word_to_index["<unk>"] = 3

index_to_word[0] = "<pad>"
index_to_word[1] = "<start>"
index_to_word[2] = "<end>"
index_to_word[3] = "<unk>"

In [33]:
for idx in range(len(aug_answer)):
    aug_answer[idx] = ["<start>"] + aug_answer[idx] + ["<end>"]

In [34]:
aug_answer[:5]

[['<start>', '하루', '가', '또', '가', '네요', '.', '<end>'],
 ['<start>', '위로', '해', '드립니다', '.', '<end>'],
 ['<start>', '여행', '은', '언제나', '좋', '죠', '.', '<end>'],
 ['<start>', '여행', '은', '언제나', '좋', '죠', '.', '<end>'],
 ['<start>', '눈살', '이', '찌푸려', '지', '죠', '.', '<end>']]

In [35]:
word_to_index["계획"]

999

In [36]:
def word_to_vector(token, word_to_index) :
    if token in word_to_index :
        return word_to_index[token]
    return word_to_index["<unk>"]

def vector_to_word(token, index_to_word) :
    if token in index_to_word :
        return index_to_word[token]
    return index_to_word[3]

def word_to_vectors(tokens, word_to_index) :
    corpus = []
    for token in tokens :
        corpus.append(word_to_vector(token,word_to_index))
    return corpus

def vector_to_words(tokens, index_to_word) :
    corpus = []
    for token in tokens :
        corpus.append(vector_to_word(token,index_to_word))
    return corpus

In [38]:
src_corpus = []

for tokens in aug_question:
    src_corpus.append(word_to_vectors(tokens,word_to_index))
    
src_corpus[:3]

[[2671, 185, 4047, 91],
 [297, 4048, 532, 1121, 11],
 [323, 2364, 593, 61, 224, 246, 10, 12, 36, 32]]

In [40]:
tgt_corpus = []

for tokens in aug_answer:
    tgt_corpus.append(word_to_vectors(tokens,word_to_index))
    
tgt_corpus[:3]

[[1, 269, 10, 154, 10, 37, 4, 2],
 [1, 523, 15, 1504, 4, 2],
 [1, 250, 18, 633, 13, 34, 4, 2]]

In [42]:
from sklearn.model_selection import train_test_split

enc_tensor = tf.keras.preprocessing.sequence.pad_sequences(src_corpus, padding='post')
dec_tensor = tf.keras.preprocessing.sequence.pad_sequences(tgt_corpus, padding='post')

enc_train, enc_val, dec_train, dec_val = \
train_test_split(enc_tensor, dec_tensor, test_size=0.01)

print("enc_train :", len(enc_train), "enc_val :", len(enc_val))
print("dec_train :", len(dec_train), "dec_val :",len(dec_val))

enc_train : 30927 enc_val : 313
dec_train : 30927 dec_val : 313


In [43]:
len(enc_train[0])

19

### 트랜스포머 구현

In [44]:
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_causality_mask(src_len, tgt_len):
    mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
    return tf.cast(mask, tf.float32)

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_padding_mask(tgt)

    dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
    dec_mask = tf.maximum(dec_mask, dec_causality_mask)

    dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
    dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)

    return enc_mask, dec_enc_mask, dec_mask
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions
        

    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x

    
    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)
                        
        out = self.combine_heads(out)
        out = self.linear(out)
            
        return out, attention_weights
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
            
        return out
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):

        """
        Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual
        
        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, enc_attn
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, causality_mask, padding_mask):

        """
        Masked Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        """
        Multi-Head Attention
        """
        residual = out
        out = self.norm_2(out)
        out, dec_enc_attn = self.dec_self_attn(out, enc_out, enc_out, causality_mask)
        out = self.do(out)
        out += residual
        
        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
    
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
        
        return out, enc_attns
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
                            
    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, causality_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out

        
    def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        
        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, causality_mask, dec_mask)
        
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns

In [45]:
VOCAB_SIZE = len(word_to_index)

transformer = Transformer(
    n_layers=2,
    d_model=128,
    n_heads=8,
    d_ff=128,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.5,
    shared_fc=True,
    shared_emb=True)
		
d_model = 128

### 모델 학습

In [46]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

In [47]:
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)

In [48]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [49]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

In [52]:
# Attention 시각화 함수

def visualize_attention(src, tgt, enc_attns, dec_attns, dec_enc_attns):
    def draw(data, ax, x="auto", y="auto"):
        import seaborn
        seaborn.heatmap(data, 
                        square=True,
                        vmin=0.0, vmax=1.0, 
                        cbar=False, ax=ax,
                        xticklabels=x,
                        yticklabels=y)
        
    for layer in range(0, 2, 1):
        fig, axs = plt.subplots(1, 4, figsize=(20, 10))
        print("Encoder Layer", layer + 1)
        for h in range(4):
            draw(enc_attns[layer][0, h, :len(src), :len(src)], axs[h], src, src)
        plt.show()
        
    for layer in range(0, 2, 1):
        fig, axs = plt.subplots(1, 4, figsize=(20, 10))
        print("Decoder Self Layer", layer+1)
        for h in range(4):
            draw(dec_attns[layer][0, h, :len(tgt), :len(tgt)], axs[h], tgt, tgt)
        plt.show()

        print("Decoder Src Layer", layer+1)
        fig, axs = plt.subplots(1, 4, figsize=(20, 10))
        for h in range(4):
            draw(dec_enc_attns[layer][0, h, :len(tgt), :len(src)], axs[h], src, tgt)
        plt.show()

In [53]:
def evaluate(sentence, model, tokenizer):
    mecab = Mecab()
    sentence = preprocess_sentence(sentence)    

    pieces = mecab.morphs(sentence)
    tokens = word_to_vectors(pieces, word_to_index)

    _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')
    ids = []
    output = tf.expand_dims([word_to_index["<start>"]], 0)
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(_input, 
              output,
              enc_padding_mask,
              combined_mask,
              dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if word_to_index["<end>"] == predicted_id:
            result = vector_to_words(ids, index_to_word)
            return pieces, result, enc_attns, dec_attns, dec_enc_attns

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)
        

    result = vector_to_words(ids, index_to_word)
    #result = tokenizer.decode_ids(ids)

    return pieces, result, enc_attns, dec_attns, dec_enc_attns

In [54]:
def translate(sentence, model, tokenizer, plot_attention=False):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = \
    evaluate(sentence, model, tokenizer)
    
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    if plot_attention:
        visualize_attention(pieces, result.split(), enc_attns, dec_attns, dec_enc_attns)

In [55]:
from tqdm import tqdm_notebook 

BATCH_SIZE = 64
EPOCHS = 10

examples = [
            "지루하다, 놀러가고 싶어.",
            "오늘 일찍 일어났더니 피곤하다.",
            "간만에 여자친구랑 데이트 하기로 했어.",
            "집에 있는다는 소리야."
]

for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm_notebook(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_train[idx:idx+BATCH_SIZE],
                    dec_train[idx:idx+BATCH_SIZE],
                    transformer,
                    optimizer)

        total_loss += batch_loss
        
        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))
        
    for example in examples:
        translate(example, transformer, tokenizer)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/484 [00:00<?, ?it/s]

Input: 지루하다, 놀러가고 싶어.
Predicted translation: ['잘', '하', '는', '게', '하', '세요', '.']
Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: ['잘', '하', '는', '거', '예요', '.']
Input: 간만에 여자친구랑 데이트 하기로 했어.
Predicted translation: ['저', '은', '이', '있', '어요', '.']
Input: 집에 있는다는 소리야.
Predicted translation: ['저', '은', '이', '있', '어요', '.']


  0%|          | 0/484 [00:00<?, ?it/s]

Input: 지루하다, 놀러가고 싶어.
Predicted translation: ['좋', '은', '사람', '이', '있', '어요', '.']
Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: ['마음', '을', '더', '더', '더', '더', '더', '더', '더', '더', '더', '더', '더', '더', '더', '더', '더', '잘', '예요', '.']
Input: 간만에 여자친구랑 데이트 하기로 했어.
Predicted translation: ['마음', '에', '는', '사람', '이', '있', '을', '거', '예요', '.']
Input: 집에 있는다는 소리야.
Predicted translation: ['친구', '에', '좋', '은', '곳', '이', '에요', '.']


...

Input: 지루하다, 놀러가고 싶어.
Predicted translation: ['마음', '을', '먹', '어도', '놀', '다', '보', '세요', '.']
Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: ['눈', '을', '더', '일찍', '덜', '덜', '덜', '덜', '덜', '아프', '답니다', '.']
Input: 간만에 여자친구랑 데이트 하기로 했어.
Predicted translation: ['저', '도', '요', '.']
Input: 집에 있는다는 소리야.
Predicted translation: ['내일', '이', '많', '은가', '봐요', '.']


  0%|          | 0/484 [00:00<?, ?it/s]

Input: 지루하다, 놀러가고 싶어.
Predicted translation: ['시간', '과', '함께', '도', '살', '수', '있', '을', '거', '예요', '.']
Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: ['이제', '판단', '먹', '든', '주무세요', '.']
Input: 간만에 여자친구랑 데이트 하기로 했어.
Predicted translation: ['마음', '도', '생각날']
Input: 집에 있는다는 소리야.
Predicted translation: ['언젠가', '군요', '.']


  0%|          | 0/484 [00:00<?, ?it/s]

Input: 지루하다, 놀러가고 싶어.
Predicted translation: ['시간', '과', '잘', '가요', '.']
Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: ['이제', '생각', '해서', '던', '눈', '을', '바뀌', '봐요', '.']
Input: 간만에 여자친구랑 데이트 하기로 했어.
Predicted translation: ['기분', '이', '많', '은', '커피', '까지', '좋', '겠', '어요', '.']
Input: 집에 있는다는 소리야.
Predicted translation: ['시간', '이', '흘렀', '네요', '.']


## 회고
데이터를 불러오는 데부터 시작해서 전처리 하는 과정을 거치면서 이전 단계에서 했던 내용들을 복습할 수 있어서 좋았다. 하지만 아직 구현하는 방법을 완전히 이해한 것은 아니어서 다른 사람들의 코드를 통해 도움을 얻은 점이 아쉽다.

실험 결과를 봤을 땐 epochs가 낮을 때 오히려 자연스러운 대답이 나온 경우가 있어 학습이 부족했단 생각이 들었다. 조사의 쓰임에 있어서도 어색한 부분이 있어서 이 부분을 보완한다면 좀 더 나은 챗봇이 될 거라고 생각했다. 